In [ ]:
import config, src
import numpy as np
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
data = src.load_data(config.PROCESSED_DATA_COMBINED_RAP_HIPHOP_FILE)

X_train, X_test, y_train, y_test = src.split_data(data, target_column='music_genre')

class_weight = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight = dict(zip(np.unique(y_train), class_weight))

Données chargées avec succès.


Dans cette partie on se propose d'appliquer les méthodes de classifications de types séquentielles du Boosting. Les méthodes de Boosting remontent á l'article de Friedman sur l'approximation de fonctions "GREEDY" par une optimisation numérique de fonctions. Divers algorithmes tels que AdaBOOST ,XGBOOST ont été developpés ces derniéres années et se sont révéles particulièrement efficaces. On utilise l' estimateur  HistGradientBoostingClassifier car il a l'avantage d'être plus rapide que GradientBoostingClassifier\(\).  



In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

Gbd=HistGradientBoostingClassifier(learning_rate=0.1,max_iter=100)

Gbd.fit(X_train,y_train)
y_pred = Gbd.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

On réalise une GridSearch pour améliorer nos résultats.

In [ ]:

from sklearn.metrics import make_scorer, accuracy_score
from sklearn.experimental import enable_hist_gradient_boosting  
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {
    'learning_rate': np.arange(0.05, 1, 0.2),
    'max_iter': np.arange(50,200,10)
}


Gbd = HistGradientBoostingClassifier()

scoring = make_scorer(accuracy_score, greater_is_better=True)


grid_search = GridSearchCV(Gbd, param_grid, cv=5, scoring=scoring)
grid_search.fit(X_train, y_train)


scores = -grid_search.cv_results_['mean_test_score']
scores = scores.reshape(len(param_grid['max_iterations']), len(param_grid['learning_rate']))

In [ ]:
plt.figure(figsize=(10, 6))
for i, lr in enumerate(param_grid['learning_rate']):
    plt.plot(param_grid['max_iterations'], scores[i], label=f'learning Rate: {lr}')

plt.title('Grid Search Scores')
plt.xlabel('Max Iterations')
plt.ylabel('Negative Mean Test Score')
plt.legend()
plt.show()


best_params = grid_search.best_params_
print("Best Parameters:", best_params)